In [2]:
%tensorflow_version 1.x
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from pathlib import Path
import os
import keras
os.chdir('/content/drive/My Drive/haikurnn-master/haikurnn-master/notebooks/models/v1')
tf_session = tf.Session()
from keras import backend as K
K.set_session(tf_session)
import math
from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
import requests
import numpy as np
import pandas as pd
from sklearn.externals import joblib

# Load Input

In [ ]:
# Settings

# Percent of samples to use for training, might be necessary if you're running out of memory
sample_size = 1

# The latent dimension of the LSTM
latent_dim = 4096

# Number of epochs to train for
epochs = 40

name = 'test'
output_dir = Path('output_%s' % name)
output_dir.mkdir()

In [ ]:
f1 = open("first.txt","r")
f2 = open("second.txt","r")
f3 = open("third.txt","r")

In [ ]:
first = f1.read().split(sep='\n')
second = f2.read().split(sep='\n')
third = f3.read().split(sep='\n')

In [ ]:
for x in first:
  if len(x) < 2:
    first.remove(x)
for x in second:
  if len(x) < 2:
    second.remove(x)
for x in third:
  if len(x) < 2:
    third.remove(x)

In [ ]:
print(len(first), len(second), len(third))
first = first[:4744]
print(len(first), len(second), len(third))


In [ ]:
df = pd.DataFrame()
df['0'] = first[0::2]
df['1'] = second[0::2]
df['2'] = third[0::2]
df['3'] = first[1::2]
df['4'] = second[1::2]
df['5'] = third[1::2]
df['0_syllables'] = [11 for x in range(2372)]
df['1_syllables'] = [11 for x in range(2372)]
df['2_syllables'] = [11 for x in range(2372)]
df['3_syllables'] = [11 for x in range(2372)]
df['4_syllables'] = [11 for x in range(2372)]
df['5_syllables'] = [11 for x in range(2372)]
df

# Format Input for Training

In [ ]:
# Drop samples that are longer that the 99th percentile of length

max_line_length = int(max([df['%s' % i].str.len().quantile(.99999) for i in range(6)]))
df = df[
    (df['0'].str.len() <= max_line_length) & 
    (df['1'].str.len() <= max_line_length) & 
    (df['2'].str.len() <= max_line_length) &
    (df['3'].str.len() <= max_line_length) & 
    (df['4'].str.len() <= max_line_length) & 
    (df['5'].str.len() <= max_line_length)
].copy()
df

In [ ]:
# Pad the lines to the max line length with new lines
for i in range(6):
    # For input, duplicate the first character
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_length+2, 'right', '\n')
    
    if i == 5: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_length+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_length+2, 'right', '\n')
    
max_line_length += 2

df

In [ ]:
inputs = df[['0_in', '1_in', '2_in', '3_in', '4_in', '5_in']].values

tokenizer = Tokenizer(filters='', char_level=True)
tokenizer.fit_on_texts(inputs.flatten())
n_tokens = len(tokenizer.word_counts) + 1

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
    tokenizer.texts_to_sequences(inputs[:,i]) for i in range(6)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out', '3_out', '4_out', '5_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    tokenizer.texts_to_sequences(outputs[:,i]) for i in range(6)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables', '3_syllables', '4_syllables', '5_syllables']].values

In [ ]:
joblib.dump([latent_dim, n_tokens, max_line_length, tokenizer], str(output_dir / 'metadata.pkl'))

# Training Model

In [ ]:
class TrainingLine:
    def __init__(self, name, previous_line, lstm, lstm1, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        #self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)

        if previous_line:
            initial_state = [
                Add(name='add_h_%s' % name)([
                    previous_line.lstm_h,
                    self.syllable_dense_output
                ]),
                Add(name='add_c_%s' % name)([
                    previous_line.lstm_c,
                    self.syllable_dense_output
                ])
            ]
        else:
            initial_state = [self.syllable_dense_output, self.syllable_dense_output]

        self.lstm_out, self.lstm_h, self.lstm_c = lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

def create_training_model(latent_dim, n_tokens):
    lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm')
    lstm1 = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm')
    lines = []
    inputs = []
    outputs = []

    for i in range(6):
        previous_line = lines[-1] if lines else None
        lines.append(TrainingLine('line_%s' % i, previous_line, lstm, lstm1, n_tokens))
        inputs += [lines[-1].char_input, lines[-1].syllable_input]
        outputs.append(lines[-1].output)

    training_model = Model(inputs, outputs)
    training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    return training_model, lstm, lines, inputs, outputs


In [ ]:
training_model, lstm, lines, inputs, outputs = create_training_model(latent_dim, n_tokens)
print(training_model.summary())
filepath = str(output_dir / ("%s-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5" % latent_dim))
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger(str(output_dir / 'training_log.csv'), append=True, separator=',')

callbacks_list = [checkpoint, csv_logger]

training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2],
    X[3], X_syllables[:,3],
    X[4], X_syllables[:,4],
    X[5], X_syllables[:,5]
], [Y[0], Y[1], Y[2], Y[3], Y[4], Y[5]], batch_size=16, epochs=epochs, validation_split=.1, callbacks=callbacks_list)

# Test Model

In [ ]:
class GeneratorLine:
    def __init__(self, name, training_line, lstm, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        self.h_input = Input(shape=(lstm.units,), name='h_input_%s' % name)
        self.c_input = Input(shape=(lstm.units,), name='c_input_%s' % name)
        initial_state = [self.h_input, self.c_input]

        self.lstm = lstm

        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

        self.syllable_dense.set_weights(training_line.syllable_dense.get_weights())
        self.output_dense.set_weights(training_line.output_dense.get_weights())

def sample(preds, temperature=1):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class Generator:
    def __init__(self, lstm, lines, tf_session, tokenizer, n_tokens, max_line_length):
        self.tf_session = tf_session
        self.tokenizer = tokenizer
        self.n_tokens = n_tokens
        self.max_line_length = max_line_length

        self.lstm = LSTM(
            lstm.units, return_state=True, return_sequences=True,
            name='generator_lstm'
        )
        self.lstm1 = LSTM(
            lstm.units, return_state=True, return_sequences=True,
            name='generator_lstm'
        )
        self.lines = [
            GeneratorLine(
                'generator_line_%s' % i,
                lines[i], self.lstm, self.n_tokens
            ) for i in range(6)
        ]
        self.lstm.set_weights(lstm.get_weights())
        self.lstm1.set_weights(lstm1.get_weights())

    def generate(self, syllables=[11, 11, 11, 11, 11, 11], temperature=1, first_char=None):
        output = []
        h = None
        c = None

        if first_char is None:
            first_char = chr(int(np.random.randint(ord('a'), ord('z')+1)))

        next_char = self.tokenizer.texts_to_sequences(first_char)[0][0]

        for i in range(6):
            line = self.lines[i]
            s = self.tf_session.run(
                line.syllable_dense_output,
                feed_dict={
                    line.syllable_input: [[syllables[i]]]
                }
            )

            if h is None:
                h = s
                c = s
            else:
                h = h + s
                c = c + s

            line_output = [next_char]
            tokens = self.n_tokens
            end = False
            next_char = None
            for i in range(self.max_line_length):
                char, h, c = self.tf_session.run(
                    [line.output, line.lstm_h, line.lstm_c],
                    feed_dict={
                        line.char_input: [[
                            np_utils.to_categorical(
                                line_output[-1],
                                num_classes=self.n_tokens
                            )
                        ]],
                        line.h_input: h,
                        line.c_input: c
                    }
                )

                char = sample(char[0,0], temperature)
                if char == 1 and not end:
                    end = True
                if char != 1 and end:
                    next_char = char
                    char = 1
                line_output.append(char)
            
            cleaned_text = self.tokenizer.sequences_to_texts([
                line_output
            ])[0].strip()[1:].replace(
                '   ', '\n'
            ).replace(' ', '').replace('\n', ' ')

            print(cleaned_text)
            output.append(cleaned_text)
        print('\n')
        return output

In [ ]:
for boh in range(30):
  generator = Generator(lstm, lines, tf_session, tokenizer, n_tokens, max_line_length)
  generator.generate()